In [1]:
import tensorflow as tf
import time
import numpy as np
import matplotlib.pyplot as plt

mnist = tf.keras.datasets.mnist
to_categorical = tf.keras.utils.to_categorical
Sequential = tf.keras.models.Sequential
Conv2D = tf.keras.layers.Conv2D
MaxPooling2D = tf.keras.layers.MaxPooling2D
Flatten = tf.keras.layers.Flatten
Dense = tf.keras.layers.Dense
Dropout = tf.keras.layers.Dropout

In [2]:
# Load dataset

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Preprocess dataset
train_images: np.ndarray = train_images.reshape((60000, 28*28, 1)).astype('float32') / 255
test_images: np.ndarray = test_images.reshape((10000, 28*28, 1)).astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [3]:
def create_model():
    model = Sequential()
    model.add(Dense(784, activation='relu', input_shape=(28*28,)))  # One neuron per pixel
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_model_faster():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
# Function to train model and measure time
def train_model(device):
    with tf.device(device):
        model = create_model()
        start_time = time.time()
        model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_split=0.3)
        end_time = time.time()
    return model, end_time - start_time

# Train on CPU
cpu_model, cpu_time = train_model('/CPU:0')
print(f'Training time on CPU: {cpu_time} seconds')

# Train on GPU
if tf.config.list_physical_devices('GPU'):
    gpu_model, gpu_time = train_model('/GPU:0')
    print(f'Training time on GPU: {gpu_time} seconds')
else:
    print('No GPU found.')


Epoch 1/10
657/657 [==============================] - 11s 16ms/step - loss: 1.7227 - accuracy: 0.3120 - val_loss: 1.1616 - val_accuracy: 0.4899
Epoch 2/10
657/657 [==============================] - 12s 18ms/step - loss: 0.9021 - accuracy: 0.6719 - val_loss: 0.4665 - val_accuracy: 0.8318
Epoch 3/10
657/657 [==============================] - 12s 19ms/step - loss: 0.5180 - accuracy: 0.8320 - val_loss: 0.3559 - val_accuracy: 0.8532
Epoch 4/10
657/657 [==============================] - 13s 20ms/step - loss: 0.3713 - accuracy: 0.9015 - val_loss: 0.2282 - val_accuracy: 0.9430
Epoch 5/10
657/657 [==============================] - 16s 24ms/step - loss: 0.2771 - accuracy: 0.9407 - val_loss: 0.1984 - val_accuracy: 0.9561
Epoch 6/10
657/657 [==============================] - 16s 25ms/step - loss: 0.2191 - accuracy: 0.9547 - val_loss: 0.1965 - val_accuracy: 0.9567
Epoch 7/10
657/657 [==============================] - 16s 24ms/step - loss: 0.1917 - accuracy: 0.9610 - val_loss: 0.1806 - val_accuracy:

In [ ]:
def display_predictions(model):
    # Predict on test images
    predictions = model.predict(test_images)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(test_labels, axis=1)

    # Identify correctly and incorrectly classified images
    correct_indices = np.where(predicted_labels == true_labels)[0]
    incorrect_indices = np.where(predicted_labels != true_labels)[0]

    # Display first 5 correct predictions
    print("First 5 Correct Predictions:")
    plt.figure(figsize=(10, 5))
    for i, idx in enumerate(correct_indices[:5]):
        plt.subplot(1, 5, i + 1)
        plt.imshow(test_images[idx].reshape(28, 28), cmap='gray')
        plt.title(f"Pred: {predicted_labels[idx]}\nTrue: {true_labels[idx]}")
        plt.axis('off')
    plt.show()

    # Display first 5 incorrect predictions
    print("First 5 Incorrect Predictions:")
    plt.figure(figsize=(10, 5))
    for i, idx in enumerate(incorrect_indices[:5]):
        plt.subplot(1, 5, i + 1)
        plt.imshow(test_images[idx].reshape(28, 28), cmap='gray')
        plt.title(f"Pred: {predicted_labels[idx]}\nTrue: {true_labels[idx]}")
        plt.axis('off')
    plt.show()

# Display predictions for CPU model
print("CPU Model Predictions:")
display_predictions(cpu_model)

# Display predictions for GPU model (if available)
if tf.config.list_physical_devices('GPU'):
    print("GPU Model Predictions:")
    display_predictions(gpu_model)